In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skfdiff import Model, Simulation
import pylab as pl
import numpy as np
from scipy.signal.windows import gaussian

### Define PDE and domain

$\frac{\partial u}{\partial t}+ u \frac{\partial u}{\partial x} = v \frac{\partial^2 u}{\partial x},\quad x\in[-1,1],\text{ }t\in[0,1]$

##### Boundary and initial conditions

$u(-1,t)=u(1,t)=0$

$u(x,0)=-sin(\pi x)$

In [ ]:
bc = {"u": {"x": ("0", "0")}}

model = Model("-u * dxu + v * dxxu",
              unknowns="u(x)",
              parameters="v",
              boundary_conditions=bc
)



order: determines grid distance h, which is halfed with each order

In [ ]:
order = 7
time_step = 0.01/(2**order)
n_xs = 255*(2**order)+1
print(f"mesh: {time_step}* {1/n_xs}]")


# Interpolation

In [ ]:
# interpolation
x = np.concatenate([np.array([-1]),
                    np.arange(-1,1,2/(n_xs-1)) + 1/(n_xs-1),
                    np.array([1])])
print(f"indre gitter størrelse: {len(np.arange(-1,1,2/(n_xs-1)))}")
U = -np.sin(np.pi*x)
v = 0.01/np.pi
initial_fields = model.Fields(x=x, u=U, v=v)
simulation = Simulation(model, initial_fields, dt=time_step/2, tmax=1, id="test")
initial_fields["u"].plot()

In [ ]:
1/time_step

In [ ]:
import time
container = simulation.attach_container()
start = time.time()
tmax, final_fields = simulation.run()
stop =  time.time() - start
print(stop)

In [ ]:
data=container.data
z = data["u"].to_numpy()
z.shape

In [ ]:
# order 3
filt_x = np.array([True] + ([True]+ [False]*(2**order-1) )*255 + 2*[True])
filt_t = np.array( ([True]+ [False]*(2**(order-1)-1) )*100 + [False])
print(filt_x.shape)
print(filt_t.shape)
data=container.data
z = data["u"].to_numpy()
print(z.shape)
z = z[:,filt_x]
z = z[filt_t,:]
print(z.shape)

for t in range():
    pass

# On Grid 

In [ ]:
# on grid
x = np.linspace(-1, 1, n_xs -1) + 1/(n_xs*2) 
U = -np.sin(np.pi*x)
v = 0.01/np.pi
initial_fields = model.Fields(x=x, u=U, v=v)
simulation = Simulation(model, initial_fields, dt=time_step, tmax=1, id="test")
initial_fields["u"].plot()

In [ ]:
import time
container = simulation.attach_container()
start = time.time()
tmax, final_fields = simulation.run()
stop =  time.time() - start
print(stop)

In [ ]:
filt_x = np.array( ([True]+ [False]*(2**order-1) )*255 + [True])
filt_t = np.array( ([True]+ [False]*(2**order-1) )*100 + [False])

data=container.data
z = data["u"].to_numpy()
print(z.shape)
z = z[:,filt_x]
z = z[filt_t,:]
print(z.shape)

t = np.linspace(0,1,101)[:-1]
x = np.linspace(-1,1,256)
xx,tt = np.meshgrid(x,t)

_, y_true = gen_testdata()
err = np.abs(z - y_true)
max_err = err.max()
avg_err = err.mean()
print(f"max error: {max_err},\n average error: {avg_err}")

In [ ]:
filt_x = np.array([True, False]*255+[True])
filt_t = np.array([True, False]*100+[False])

data=container.data
z = data["u"].to_numpy()
z = z[:,filt_x]
z = z[filt_t,:]
print(z.shape)

t = np.linspace(0,1,101)[:-1]
x = np.linspace(-1,1,256)
xx,tt = np.meshgrid(x,t)

_, y_true = gen_testdata()
err = np.abs(z - y_true)
max_err = err.max()
avg_err = err.mean()
max_err, avg_err

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(16, 6)

cs0 = ax1.contourf(xx,tt,z, levels=1000, cmap="rainbow")
fig.colorbar(cs0, ax=ax1, label=r"$u(x,t)$")
ax2.set_title("FDM prediction", fontsize=16)
ax2.set_xlabel("x", fontsize=14)
ax2.set_tlabel("t", fontsize=14)


cs1 = ax2.contourf(xx,tt,err, levels=1000, cmap="rainbow")
ax2.set_title("FDM error", fontsize=16)
ax2.set_xlabel("x", fontsize=14)
ax2.set_tlabel("t", fontsize=14)

fig.colorbar(cs1, ax=ax2, label="absolute error")

plt.savefig("FDM_burgers_err.png",
           facecolor="white")


In [ ]:
def gen_testdata():
    data = np.load("Burgers.npz")
    t, x, exact = data["t"], data["x"], data["usol"].T
    print(t.shape,x.shape)
    xx, tt = np.meshgrid(x,t)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = exact
    return X, y

In [ ]:

_, y_true = gen_testdata()

# FDM theory Example

In [ ]:
b = np.array([2,0,0,2*np.exp(3*2)])
#b = np.zeros(5)
A = np.zeros((4,4))
for i in range(1,3):
    A[i,i] = -15/2
    A[i,i-1] = 3
    A[i,i+1] = 3/2
    
A[0,0] = 1
A[3,3] = 1

y = np.linalg.solve(A, b)

In [ ]:
b2 = np.array([2,0,0,0,0,0,2*np.exp(3*2)])
#b = np.zeros(5)
A2 = np.zeros((7,7))
for i in range(1,6):
    A2[i,i+1] = 6 
    A2[i,i]   = -21
    A2[i,i-1] = 12
    
A2[0,0] = 1
A2[6,6] = 1

y2 = np.linalg.solve(A2, b2)

In [ ]:
x = np.array([0, 2/3, 4/3, 2])
x2 = np.arange(0,2.01,1/3)
xs = np.linspace(0,2,1001)
f = lambda x: 2*np.exp(3*x)

In [ ]:
x2

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.tight_layout()
plt.scatter(x,y, label="FDM, $h=2/3$")
plt.scatter(x2,y2,label="FDM, $h=1/3$", color="red")
plt.plot(xs,f(xs), label="Latent function: $2e^{3x}$", color="orange", linestyle="--")
plt.legend(fontsize=14)
plt.title("FDM approximation")
plt.xlabel("x", fontsize=12)
plt.ylabel("f(x)", fontsize=12)
#plt.yscale("log")
plt.savefig("/Users/martin/Library/Mobile Documents/com~apple~CloudDocs/Uni/BA/figurer/FDM_approx.png")

In [ ]:
result = np.arange(0,5,1)